In [1]:
using AdaptiveDistanceFields
using RegionTrees: allleaves, vertices
using StaticArrays
using Plots
gr()

Plots.GRBackend()

In [2]:
function curve_distance(x)
    # line from [0.2, 0.2] to [0.2, 0.8]
    # arc centered at [0.2, 0.5] with radius 0.3
    
    if x[2] < 0.2
        distance_to_line = norm(x - SVector(0.2, 0.2))
    elseif x[2] > 0.8
        distance_to_line = norm(x - SVector(0.2, 0.8))
    else
        distance_to_line = norm(x[1] - 0.2)
    end
    if x[1] >= 0.2
        distance_to_arc = norm(norm(x - SVector(0.2, 0.5)) - 0.3)
    else
        distance_to_arc = Inf
    end
        
    min(distance_to_arc, distance_to_line)
end
        

curve_distance (generic function with 1 method)

In [7]:
x = linspace(0, 1, 101)
y = linspace(0, 1, 101)
contour(x, y, (x, y) -> curve_distance(SVector(x, y)), fill=true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
<image width="18" height="381" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAAF9CAYAAAAJGzhsAAABBElEQVR4nO3cMWpCURCG0fd0Ci1i
I8E1uG/3kFW4gkAkxoCCICh5hYhJ/8rLFEM4/wIOfNUUF27/eG5+u4RNMhAQCAQCgf4bFMP3WwpU
Lw0EAoFAoPbF7GObAtVLA4FAIBCoABT9+y4FqpcGAoFAIFD74v65SIHqpYFAIBAI1L647lcpUL00
EAgEAoEKQHE+vaZA9dJAIBAIBGpfHC/LFKheGggEAoFABaD4unkVBYFAIBBovDj8zFOgemkgEAgE
ArUv9sM0BaqXBgKBQCBQASgOwyMFqpcGAoFAIFD7YtcdU6B6aSAQCAQCFYD6l9naF/MgEAgEAo3W
d93UgQSBQCAQaLQ/Fiwg0qpwdswAAAAASUVORK5CYII=
" transform="translate(548, 3)"/>
 
 
 0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6

In [4]:
adf = AdaptiveDistanceField(curve_distance, SVector(0., 0), SVector(1., 1))

(::AdaptiveDistanceField) (generic function with 1 method)

In [6]:
x = linspace(0, 1)
y = linspace(0, 1)
plt = contour(x, y, (x, y) -> adf(SVector(x, y)), fill=true, legend=nothing)
for leaf in allleaves(adf.root)
    v = hcat(collect(vertices(leaf.boundary))...)
    plot!(plt, v[1,[1,2,4,3,1]], v[2,[1,2,4,3,1]], color=:white)
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0